In [178]:
import plotly.graph_objects as go 
import pandas as pd
import googlemaps
import os
import dash
from dash import dcc,html
from dash.dependencies import Input, Output


In [80]:
data = pd.read_csv('C:\\Users\\aport\\OneDrive\\Documents\\School\\Fall Semester 2024\\Telling Stories w Data\\dashboard-assignment\\query-results.txt'
                   ,sep = '\t', on_bad_lines = 'skip')

In [81]:
data

,state,location,time_posted,name,make,model,year,odometer,price,predicted_price,...,drive,cylinders,fuel,type,transmission,condition,title_text,latitude,longitude,url
0,Oregon,portland,1.728131e+09,mazda,mazda,mazda2,2013.0,93000.0,7600.0,5694.983029,...,NaN,4 cylinders,gas,NaN,manual,good,Mazda2,45.115700,-123.174400,https://portland.craigslist.org/yam/cto/d/amit...
1,Washington,seattle,1.727539e+09,Volkswagen EuroVan MV,volkswagen,eurovan,2001.0,173123.0,11900.0,13790.984439,...,fwd,6 cylinders,gas,van,automatic,good,2001 Volkswagen Eurovan MV,47.737700,-122.343100,https://seattle.craigslist.org/see/cto/d/seatt...
2,Oregon,portland,1.727169e+09,kia soul,kia,soul,2014.0,39809.0,9999.0,7597.268923,...,NaN,NaN,gas,NaN,automatic,good,2014 Kia Soul *39k miles*,45.457300,-122.630100,https://portland.craigslist.org/wsc/cto/d/port...
3,Oregon,portland,1.727092e+09,fiat 500 pop,fiat,500,2013.0,117446.0,5250.0,5453.623778,...,fwd,4 cylinders,gas,hatchback,manual,excellent,2013 Fiat 500 Pop,45.435200,-122.715000,https://portland.craigslist.org/mlt/cto/d/port...
4,Washington,seattle,1.727082e+09,scion xb,scion,xb,2004.0,225970.0,2900.0,3041.155880,...,fwd,4 cylinders,gas,NaN,manual,good,2004 Scion Xb,47.648800,-122.396400,https://seattle.craigslist.org/see/cto/d/seatt...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15332,Washington,kpr,1.723552e+09,kia optima,kia,optima,2013.0,103000.0,7000.0,7070.918241,...,fwd,4 cylinders,gas,sedan,automatic,excellent,Kia Optima,46.384442,-118.943862,https://kpr.craigslist.org/cto/d/eltopia-kia-o...
15333,Oregon,eugene,1.723373e+09,Mercedes Benz SL550,mercedes-benz,sl,2008.0,66000.0,20000.0,17736.063509,...,rwd,8 cylinders,gas,convertible,automatic,excellent,Beautiful Mercedes Benz SL550 Hard top Convert...,44.068200,-123.081900,https://eugene.craigslist.org/cto/d/eugene-bea...
15334,Oregon,portland,1.723024e+09,ford econoline cargo van,ford,e150,2010.0,155000.0,7900.0,7595.544453,...,rwd,8 cylinders,gas,van,automatic,good,2010 Ford Econoline Cargo van E150,45.437300,-122.614700,https://portland.craigslist.org/clc/cto/d/port...
15335,Idaho,boise,1.722954e+09,pontiac grand am,pontiac,grand am,2001.0,122000.0,2800.0,NaN,...,NaN,NaN,gas,NaN,automatic,NaN,2001 Pontiac Grand Am GT.,43.706900,-116.362000,https://boise.craigslist.org/cto/d/eagle-2001-...


In [82]:
for state in data['state']:
    if state not in ['Wyoming','Montana','Oregon','Idaho','Washington']:
        print(state)

Sport Utility 4D


In [83]:
data.loc[data['city'] == 'Sport Utility 4D']

,state,location,time_posted,name,make,model,year,odometer,price,predicted_price,...,drive,cylinders,fuel,type,transmission,condition,title_text,latitude,longitude,url
12433,Sport Utility 4D,46.6054,-120.4224,https://yakima.craigslist.org/cto/d/yakima-200...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
data.drop(12433,axis = 0,inplace= True)
data.reset_index(inplace=True)

In [124]:
gmaps = googlemaps.Client(os.getenv('googlemaps'))

In [86]:
app = dash.Dash()

In [205]:
data['location'] = data['location'].str.title()

data.loc[data['location'] == 'Kpr','location'] = 'Tri-Cities'

In [206]:
unique_options = data.drop_duplicates(subset = ['location','state'])[['location','state']]

In [226]:
dropdown_options = [{'label': f"{loc}, {state}", 'value': f"{loc}, {state}"} for loc, state in unique_options.values]

In [208]:
app.layout = html.Div([
    html.H1(children='Car Search Dashboard'),
            dcc.Dropdown(id = 'geo-dropdown',
                       options = dropdown_options,
                       value = 'Missoula, Montana'),

            dcc.Graph(id = 'price-graph')
])

In [209]:
if __name__ == '__main__':
    app.run_server()

In [214]:
gmaps.geolocate()['location']['lat']
gmaps.geolocate()['location']['lng']

-73.9859414

In [174]:
gmaps.reverse_geocode((47.658400,-117.156800))[0]['formatted_address'].split(',')[1]

' Spokane Valley'

In [176]:
@app.callback(
    Output(component_id= 'price-graph', component_property= 'figure'),
    Input(component_id='geo-dropdown', component_property= 'value')
)

SyntaxError: incomplete input (3231678193.py, line 4)

In [ ]:
def update_graph(selected_state,selected_location):
    if selected_state in unique_options and selected_location in unique_options:
        filtered_cars = data[data['state'] == selected_state]
        